# Librairies and set up

In [185]:
#pip install os
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [19]:
os.getcwd()

'/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd'

In [20]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [99]:
os.chdir('/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd')

# Read the data

In [54]:
header_season2_bids=[
    'Bid ID'
    ,'Timestamp'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Advertiser ID'
    ,'User Profile IDs']

header_season2=[
    'Bid ID'
    ,'Timestamp'
    ,'Log Type'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Paying Price'
    ,'Landing Page URL'
    ,'Advertiser ID'
    ,'User Profile IDs']

bids=pd.read_table('bid.20130606.txt', names=header_season2_bids)
impressions=pd.read_table('imp.20130606.txt', names=header_season2)
clicks=pd.read_table('clk.20130606.txt', names=header_season2)
conversions=pd.read_table('conv.20130606.txt', names=header_season2)




/Users/Armand/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Data Exploration

Let's see if a record in conversions exists in clicks, impressions and bids

In [102]:
conversions.head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
0,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000132000,3,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


In [103]:
clicks[clicks['Bid ID']=='690e543c46a6cbe8bdd062f53a8ac7a2']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
8,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000125692,2,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


In [104]:
impressions[impressions['Bid ID']=='690e543c46a6cbe8bdd062f53a8ac7a2']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
15662,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000111948,1,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,71,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


Cool ! looks like the logic is OK

# Duplicates handling 

In [106]:
bids['Bid ID'].duplicated().sum()

0

No duplicated Bids based on 'Bid ID' which is good news! 

## Bids --> Impression duplicates

When trying to join the data, we see that impression bid IDs are not unique. Let's see which ones are duplicated

In [74]:
impressions['Bid ID'].duplicated().sum()

6275

... let's find the records assciated with these 

In [83]:
impressions[impressions['Bid ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
249,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105378,1,Vhk7ZAnyPQq2Xb2,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN


In [84]:
impressions[impressions['Bid ID']=='8678cd6a8dc3c8cda78a332db4e65e26']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
199,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105295,1,Vh1T1uqVL6Mfwgc,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN
249,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105378,1,Vhk7ZAnyPQq2Xb2,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN


different ipinyou id --> maybe ipinyouid is unique 

In [85]:
impressions[impressions['iPinYou ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
94,4571654093fdb94a3d2b3485e159e51e,20130606000105011,1,VhkSO71iPlM6FVc,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",175.1.175.*,201,204,2,trqRTvuYMZk7FmMs,94bee0f33b49b1c4b83906775196f4ec,NaN,86730593,300,250,1,0,5,d881a6c788e76c2c27ed1ef04f119544,238,166,d29e59bf0f7f8243858b8183f14d4412,3358,"10063,13866,10006,13800,10093,10111"


Nope, iPinYou ID is not unique. Maybe the combination of IpinYouID + Bid ID ???

In [93]:
impressions[impressions[['Bid ID','iPinYou ID']].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
5162,59088cb88db642750119b5d9eea721d7,20130606000107572,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"


In [95]:
impressions[impressions['Bid ID']=='59088cb88db642750119b5d9eea721d7']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4501,59088cb88db642750119b5d9eea721d7,20130606000107300,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"
5162,59088cb88db642750119b5d9eea721d7,20130606000107572,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"
62824,59088cb88db642750119b5d9eea721d7,20130606003107393,1,Vhk7Znl8PvNPD-C,Mozilla/4.0,221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,10063


conclusion 1 to many relationship between bids and impressions

## Impressions to clicks duplicates

In [114]:
clicks[clicks['Bid ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
6,47e7c5523cbda762a7c00e0c47c41993,20130606000123750,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


In [117]:
clicks[clicks['Bid ID']=='47e7c5523cbda762a7c00e0c47c41993']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4,47e7c5523cbda762a7c00e0c47c41993,20130606000120463,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN
6,47e7c5523cbda762a7c00e0c47c41993,20130606000123750,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


In [119]:
impressions[impressions['Bid ID']=='47e7c5523cbda762a7c00e0c47c41993']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4056,47e7c5523cbda762a7c00e0c47c41993,20130606000107128,1,Vhk7ZAn8OvpPMI2,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,113,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


Conclusion --> 1 to many relationship between impressions and clicks 

# Left join to create full dataset

In [143]:
df=pd.merge(bids, impressions, on="Bid ID", suffixes=('_bid', '_imp'), how="outer")

In [144]:
df=pd.merge(df, clicks, on="Bid ID", suffixes=('_bids_join_impressions', '_clicks'), how="outer")

In [142]:
df.head()

,Bid ID,Timestamp_bid,iPinYou ID_bid,User-Agent_bid,IP_bid,Region ID_bid,City ID_bid,Ad Exchange_bid,Domain_bid,URL_bid,Anonymous URL_bid,Ad Slot ID_bid,Ad Slot Width_bid,Ad Slot Height_bid,Ad Slot Visibility_bid,Ad Slot Format_bid,Ad Slot Floor Price_bid,Creative ID_bid,Bidding Price_bid,Advertiser ID_bid,User Profile IDs_bid,Timestamp_imp,Log Type_df,iPinYou ID_imp,User-Agent_imp,IP_imp,Region ID_imp,City ID_imp,Ad Exchange_imp,Domain_imp,URL_imp,Anonymous URL_imp,Ad Slot ID_imp,Ad Slot Width_imp,Ad Slot Height_imp,Ad Slot Visibility_imp,Ad Slot Format_imp,Ad Slot Floor Price_imp,Creative ID_imp,Bidding Price_imp,Paying Price_df,Landing Page URL_df,Advertiser ID_imp,User Profile IDs_imp,Timestamp,Log Type_clicks,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price_clicks,Landing Page URL_clicks,Advertiser ID,User Profile IDs
0,b382c1c156dcbbd5b9317cb50f6a747b,20130606000104008,Vh16OwT6OQNUXbj,mozilla/4.0 (compatible; msie 6.0; windows nt 5.1; sv1; qqdownload 718),180.127.189.*,80,87,1,tFKETuqyMo1mjMp45SqfNX,249b2c34247d400ef1cd3c6bfda4f12a,NaN,mm_11402872_1272384_3182279,300,250,1,1,0,00fccc64a1ee2809348509b7ac2a97a5,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7b6195de0d14203f92001da653bf1de,20130606000104009,Vhkr1vpROHuhQWB,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",113.119.105.*,216,217,2,trqRTuToMTNUjM9r5rMi,74419a072f8927222a1fd8aaa18cce56,NaN,433287550,468,60,1,0,5,2f88fc9cf0141b5bbaf251cab07f4ce7,300,3386,NaN,2.013061e+16,1.0,Vhkr1vpROHuhQWB,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),113.119.105.*,216.0,217.0,2.0,trqRTuToMTNUjM9r5rMi,74419a072f8927222a1fd8aaa18cce56,NaN,433287550,468.0,60.0,1.0,0.0,5.0,2f88fc9cf0141b5bbaf251cab07f4ce7,300.0,18.0,361e128affece850342293213691a043,3386.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2ea9fe21cf7350fcb5696d8cff0bbeaa,20130606000104012,VhKdLnuY3tlhXMa,mozilla/4.0 (compatible; msie 8.0; windows nt 5.1; trident/4.0; .net clr 2.0.50727),42.184.148.*,65,69,1,trqRTvpogNlyDok4JKTI,134db65c2b66d8468d00bf42fd9f912f,NaN,mm_10032051_2374052_9219530,950,90,1,1,0,23d6dade7ed21cea308205b37594003e,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104014,VhTVORqG36N6qMj,"mozilla/5.0 (windows nt 5.1) applewebkit/537.17 (khtml, like gecko) chrome/24.0.1312.57 safari/537.17",114.100.37.*,106,117,1,lsxSl559Xql7FmMs,8c9742e63497713b97ac7e780a8f9a12,NaN,mm_30232185_2681382_11190685,950,90,0,1,0,23d6dade7ed21cea308205b37594003e,227,3427,NaN,2.013061e+16,1.0,VhTVORqG36N6qMj,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",114.100.37.*,106.0,117.0,1.0,lsxSl559Xql7FmMs,8c9742e63497713b97ac7e780a8f9a12,NaN,mm_30232185_2681382_11190685,950.0,90.0,0.0,1.0,0.0,23d6dade7ed21cea308205b37594003e,227.0,207.0,b2e35064f3549d447edbbdfb1f707c8c,3427.0,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,faf17eac9cabf1be598f4e75f40d501d,20130606000104016,VhL01pk8OTkW3Mc,"mozilla/5.0 (windows nt 5.1) applewebkit/537.1 (khtml, like gecko) chrome/21.0.1180.89 safari/537.1",58.100.240.*,94,95,1,tK1NTu1YP5scFsf,e22930480589abcc1468854cb3403314,NaN,mm_10075660_3500949_11453278,950,90,0,1,0,d5cecca9a6cbd7a0a48110f1306b26d1,227,3358,NaN,2.013061e+16,1.0,VhL01pk8OTkW3Mc,"Mozilla/5.0 (Windows NT 5.1) AppleWe

In [145]:
df.shape

(9593354, 67)

In [147]:
user_profile_cols = [col for col in df.columns if 'Profile' in col]

In [150]:
user_profile_cols

['User Profile IDs_bid', 'User Profile IDs_imp', 'User Profile IDs']

In [152]:
df.filter(regex='Profile')

,User Profile IDs_bid,User Profile IDs_imp,User Profile IDs
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",NaN
4,NaN,"10059,13866,10063,10111",NaN
...,...,...,...
9593349,NaN,NaN,NaN
9593350,NaN,NaN,NaN
9593351,NaN,NaN,NaN
9593352,NaN,"10059,10077,10052,10063,10006,10031,13403,13800,10110",NaN


# Use User profile 

In [220]:
mapping=pd.read_table('/Users/Armand/Capstone/ipinyou.contest.dataset/user.profile.tags.en.txt', names=('key','value'))

In [295]:
test=df[['Bid ID','User Profile IDs_imp']]

In [ ]:
s=test['User Profile IDs_imp'].str.split(pat=',')
final_to_concatenate=pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

In [ ]:
#s=test['User Profile IDs_imp'].str.split(pat=',')
#mlb = MultiLabelBinarizer()

#pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

In [286]:
type(s)

pandas.core.series.Series

In [164]:
os.getcwd()

'/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd'